# 使用自己的数据在Azure聊天完成模型中（预览版）

> 注意：openai库有更新版本可用。请参阅 https://github.com/openai/openai-python/discussions/742

这个示例展示了如何在Azure OpenAI服务模型中使用自己的数据。该功能目前处于预览阶段。

在Azure上使用您的数据可以运行支持的聊天模型，如GPT-3.5-Turbo和GPT-4，而无需训练或微调模型。在您的数据上运行模型可以让您以更高的准确性和速度进行聊天和分析数据。Azure OpenAI在您的数据上的一个关键优势是它能够定制会话AI的内容。因为模型可以访问并引用特定来源来支持其响应，答案不仅基于其预训练知识，还基于指定数据源中可用的最新信息。这些基础数据还有助于模型避免基于过时或不正确信息生成响应。

Azure OpenAI在您自己的数据上与Azure认知搜索提供了一个可定制的、预构建的知识检索解决方案，可以构建一个会话AI应用程序。要查看知识检索和语义搜索的替代方法，请查看[向量数据库](https://github.com/openai/openai-cookbook/tree/main/examples/vector_databases)的示例。


## 工作原理

[Azure OpenAI on your own data](https://learn.microsoft.com/azure/ai-services/openai/concepts/use-your-data) 将模型与您的数据连接起来，使其能够检索和利用数据，从而增强模型的输出能力。与 Azure Cognitive Search 一起，根据用户输入和提供的对话历史，从指定的数据源中检索数据。然后，对数据进行增强处理，并重新提交给模型作为提示，为模型提供可以用来生成响应的上下文信息。

有关更多信息，请参阅[Azure OpenAI 服务的数据、隐私和安全性](https://learn.microsoft.com/legal/cognitive-services/openai/data-privacy?context=%2Fazure%2Fai-services%2Fopenai%2Fcontext%2Fcontext)。


## 先决条件
为了开始，我们将介绍一些先决条件。

要正确访问Azure OpenAI服务，我们需要在[Azure门户](https://portal.azure.com)上创建适当的资源（您可以在[Microsoft Docs](https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal)中查看如何执行此操作的详细指南）

要在Azure OpenAI模型中使用您自己的数据，您将需要：

1. Azure OpenAI访问权限以及部署了聊天模型的资源（例如，GPT-3或GPT-4）
2. Azure认知搜索资源
3. Azure Blob存储资源
4. 作为数据使用的文档（请参阅[数据源选项](https://learn.microsoft.com/azure/ai-services/openai/concepts/use-your-data#data-source-options))

要了解如何将您的文档上传到Blob存储并使用Azure AI Studio创建索引的完整步骤，请参阅此[快速入门指南](https://learn.microsoft.com/azure/ai-services/openai/use-your-data-quickstart?pivots=programming-language-studio&tabs=command-line)。


## 设置

首先，我们安装必要的依赖项。


In [ ]:
! pip install "openai>=0.28.1,<1.0.0"
! pip install python-dotenv


在这个示例中，我们将使用 `dotenv` 来加载我们的环境变量。为了连接到Azure OpenAI和搜索索引，以下变量应该以 `KEY=VALUE` 的格式添加到一个名为 `.env` 的文件中：

* `OPENAI_API_BASE` - Azure OpenAI的终端节点。可以在Azure门户中的Azure OpenAI资源的“密钥和终结点”下找到。
* `OPENAI_API_KEY` - Azure OpenAI的API密钥。可以在Azure门户中的Azure OpenAI资源的“密钥和终结点”下找到。如果使用Azure Active Directory身份验证，则可以省略（参见下面的“使用Microsoft Active Directory进行身份验证”）
* `SEARCH_ENDPOINT` - Cognitive Search的终端节点。这个URL可以在Azure门户中的搜索资源的“概述”中找到。
* `SEARCH_KEY` - Cognitive Search的API密钥。可以在Azure门户中的搜索资源的“密钥”下找到。
* `SEARCH_INDEX_NAME` - 您使用自己的数据创建的索引的名称。


In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()


In [2]:
openai.api_base = os.environ["OPENAI_API_BASE"]

# Azure OpenAI 自定义数据功能仅在 2023-08-01-preview API 版本中提供支持。
openai.api_version = "2023-08-01-preview"


### 认证

Azure OpenAI 服务支持多种认证机制，包括 API 密钥和 Azure 凭据。


In [3]:
use_azure_active_directory = False  # 将此标志设置为 True，如果您正在使用 Azure Active Directory。


#### 使用API密钥进行身份验证

要设置OpenAI SDK以使用*Azure API密钥*，我们需要将`api_type`设置为`azure`，并将`api_key`设置为与您的端点关联的密钥（您可以在[Azure门户](https://portal.azure.com)的*"资源管理"*下的*"密钥和终结点"*中找到此密钥）。


In [4]:
if not use_azure_active_directory:
    openai.api_type = 'azure'
    openai.api_key = os.environ["OPENAI_API_KEY"]


#### 使用Microsoft Active Directory进行身份验证
现在让我们看看如何通过Microsoft Active Directory身份验证获取密钥。有关如何设置此功能的更多信息，请参阅[文档](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity)。


In [ ]:
! pip install azure-identity


In [5]:
from azure.identity import DefaultAzureCredential

if use_azure_active_directory:
    default_credential = DefaultAzureCredential()
    token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

    openai.api_type = "azure_ad"
    openai.api_key = token.token


令牌在一段时间内有效，之后将会过期。为了确保每个请求都发送一个有效的令牌，你可以通过连接到requests.auth来刷新一个即将过期的令牌：


In [8]:
import typing
import time
import requests

if typing.TYPE_CHECKING:
    from azure.core.credentials import TokenCredential

class TokenRefresh(requests.auth.AuthBase):

    def __init__(self, credential: "TokenCredential", scopes: typing.List[str]) -> None:
        self.credential = credential
        self.scopes = scopes
        self.cached_token: typing.Optional[str] = None

    def __call__(self, req):
        if not self.cached_token or self.cached_token.expires_on - time.time() < 300:
            self.cached_token = self.credential.get_token(*self.scopes)
        req.headers["Authorization"] = f"Bearer {self.cached_token.token}"
        return req


## 使用自己的数据完成聊天补全模型


### 设置上下文


在这个例子中，我们希望我们的模型基于Azure AI服务文档数据来生成回复。根据之前分享的[快速入门](https://learn.microsoft.com/azure/ai-services/openai/use-your-data-quickstart?tabs=command-line&pivots=programming-language-studio)，我们已经将[markdown](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/ai-services/cognitive-services-and-machine-learning.md)文件添加到了我们的搜索索引中，该文件是关于[Azure AI服务和机器学习](https://learn.microsoft.com/azure/ai-services/cognitive-services-and-machine-learning)文档页面的内容。现在模型已经准备好回答关于Azure AI服务和机器学习的问题了。


### 代码


要使用Python SDK与Azure OpenAI模型使用自己的数据进行聊天，我们首先必须设置代码以针对聊天完成扩展端点，该端点旨在与您自己的数据一起使用。为此，我们创建了一个便利函数，可以调用它来为库设置自定义适配器，该适配器将针对给定部署ID的扩展端点。


In [5]:
import requests

def setup_byod(deployment_id: str) -> None:
    """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.
    
    :param deployment_id: The deployment ID for the model to use with your own data.

    To remove this configuration, simply set openai.requestssession to None.
    """

    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):

        def send(self, request, **kwargs):
            request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
            return super().send(request, **kwargs)

    session = requests.Session()

    # 挂载一个自定义适配器，该适配器将对使用指定 `deployment_id` 的任何调用使用扩展端点。
    session.mount(
        prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
        adapter=BringYourOwnDataAdapter()
    )

    if use_azure_active_directory:
        session.auth = TokenRefresh(default_credential, ["https://cognitiveservices.azure.com/.default"])

    openai.requestssession = session


现在我们可以调用便利函数，使用我们计划用于自己数据的模型来配置SDK。


In [6]:
setup_byod("gpt-4")


通过为`dataSources`关键参数提供我们的搜索端点、密钥和索引名称，现在对模型提出的任何问题都将基于我们自己的数据。另外，将提供一个名为`context`的属性，用于显示模型用来回答问题所参考的数据。


In [7]:
completion = openai.ChatCompletion.create(
    messages=[{"role": "user", "content": "What are the differences between Azure Machine Learning and Azure AI services?"}],
    deployment_id="gpt-4",
    dataSources=[  # 驼峰式命名法是故意为之，因为这是API所期望的格式。
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": os.environ["SEARCH_ENDPOINT"],
                "key": os.environ["SEARCH_KEY"],
                "indexName": os.environ["SEARCH_INDEX_NAME"],
            }
        }
    ]
)
print(completion)


{
  "id": "65b485bb-b3c9-48da-8b6f-7d3a219f0b40",
  "model": "gpt-4",
  "created": 1693338769,
  "object": "extensions.chat.completion",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Azure AI services and Azure Machine Learning (AML) both aim to apply artificial intelligence (AI) to enhance business operations, but they target different audiences and offer different capabilities [doc1]. \n\nAzure AI services are designed for developers without machine learning experience and provide pre-trained models to solve general problems such as text analysis, image recognition, and natural language processing [doc5]. These services require general knowledge about your data without needing experience with machine learning or data science and provide REST APIs and language-based SDKs [doc2].\n\nOn the other hand, Azure Machine Learning is tailored for data scientists and involves a longer process of data co

如果您希望从模型中流式传输响应，可以传递 `stream=True` 关键字参数：


In [8]:
response = openai.ChatCompletion.create(
    messages=[{"role": "user", "content": "What are the differences between Azure Machine Learning and Azure AI services?"}],
    deployment_id="gpt-4",
    dataSources=[
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": os.environ["SEARCH_ENDPOINT"],
                "key": os.environ["SEARCH_KEY"],
                "indexName": os.environ["SEARCH_INDEX_NAME"],
            }
        }
    ],
    stream=True,
)

for chunk in response:
    delta = chunk.choices[0].delta

    if "role" in delta:
        print("\n"+ delta.role + ": ", end="", flush=True)
    if "content" in delta:
        print(delta.content, end="", flush=True)
    if "context" in delta:
        print(f"Context: {delta.context}", end="", flush=True)


Context: {
  "messages": [
    {
      "role": "tool",
      "content": "{\"citations\":[{\"content\":\"<h2 id=\\\"how-are-azure-ai-services-and-azure-machine-learning-aml-similar\\\">How are Azure AI services and Azure Machine Learning (AML) similar?.</h2>\\n<p>Both have the end-goal of applying artificial intelligence (AI) to enhance business operations, though how each provides this in the respective offerings is different..</p>\\n<p>Generally, the audiences are different:</p>\\n<ul>\\n<li>Azure AI services are for developers without machine-learning experience..</li>\\n<li>Azure Machine Learning is tailored for data scientists.\",\"id\":null,\"title\":\"Azure AI services and machine learning\",\"filepath\":\"cognitive-services-and-machine-learning.md\",\"url\":\"https://krpraticstorageacc.blob.core.windows.net/azure-openai/cognitive-services-and-machine-learning.md\",\"metadata\":{\"chunking\":\"orignal document size=1188. Scores=5.689296 and None.Org Highlight count=160.Filtering 